# Versuch 2 - Kalibrierung von Digitalkameras

## Dominik Bartsch, Stefan Ptacek, Niklas Kaiser

In [1]:
import cv2
import numpy as np

### 1. Aufnahme und Analyse eines Grauwertkeiles
#### 1.1

In [2]:
# -------- Aufgabe1.1 -------- #

cap = cv2.VideoCapture(0)

for i in range(10):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("data/dark" + str(i) + ".png", gray)

print("frame width: " + str(cap.get(3)))
print("frame height: " + str(cap.get(4)))
print("--------------------------------")
print("brightness: " + str(cap.get(10)))
print("contrast: " + str(cap.get(11)))
print("saturation: " + str(cap.get(12)))
print("--------------------------------")
print("gain: " + str(cap.get(14)))
print("exposure: " + str(cap.get(15)))
print("--------------------------------")
print("white balance: " + str(cap.get(17)))

cap.release()
#cv2.destroyAllWindows()

frame width: 640.0
frame height: 480.0
--------------------------------
brightness: 128.0
contrast: 32.0
saturation: 64.0
--------------------------------
gain: -1.0
exposure: -6.0
--------------------------------
white balance: 1.0


In [3]:
img_arr = []


def fix_noise(dark_img, light_img, img):
    img = img - dark_img
    light_img = (light_img - dark_img) / np.mean(light_img)
    img = img / light_img
    return img


def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


for i in range(10):
    img_read = cv2.imread("./data/default" + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
    img_arr.append(img_read)

img = np.mean(img_arr, axis=0, dtype=float)
print(img)
sub_arrays = []
slicing_index = []

step = 20
threshold = 20
for i in range(0, len(img[0]) - step, step):
    if abs(int(img[0, i + step]) - int(img[0, i])) > threshold:
        # print("Slicing at index: " + str(i) + " with value: "
        #       + str(img[0][i]) + " and next value: " + str(img[0][i + 1]))
        slicing_index.append(i)
        sub_arrays.append(img[:, i-99:i])

slicing_index.append(len(img[0]))  # add last index
sub_arrays.append(img[:, len(img[0]) - 98:len(img[0]) - 1])

print(slicing_index)
print(sub_arrays)

res_mean = []
res_std = []
i = 0
for x in sub_arrays:
    fixed_x = fix_noise(sub_arrays[0][:,90], sub_arrays[4][:,90], x[:, 90])
    mean = np.round(np.mean(x), 2)
    mean_fixed = np.round(np.mean(fixed_x), 2)
    res_mean.append(mean)
    std = np.round(np.std(x), 2)
    std_fixed = np.round(np.std(fixed_x), 2)
    res_std.append(std)

    
    print(f"Index: {i} , Mean: {mean}, Std: {std},  Fixed Mean: {mean_fixed}, Fixed Std: {std_fixed}")
    i += 1



# visualize sub arrays
for i in range(len(sub_arrays)):
    cv2.imshow("sub_array" + str(i), sub_arrays[i].astype(np.uint8))
cv2.waitKey(0)
#cv2.destroyAllWindows()

nan


IndexError: invalid index to scalar variable.

### 2. Aufnahme eines Dunkelbildes

In [ ]:
def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


def fix_noise(dark_img, light_img, img):
    img = img - dark_img
    light_img = (light_img - dark_img) / np.mean(light_img)
    img = img / light_img
    return img


res = fix_noise(get_image("./data/dark"), get_image("./data/light"), get_image("./data/default"))
print(res)
cv2.imshow("res", res.astype(np.uint8))

img_dark = get_image("./data/dark")
img_light = get_image("./data/light")

alpha = 3  # Contrast control (1.0-3.0)
beta = 0  # Brightness control (0-100)

adjusted_dark = cv2.convertScaleAbs(img_dark, alpha=alpha, beta=beta)
adjusted_light = cv2.convertScaleAbs(img_light, alpha=alpha, beta=beta)

cv2.imshow('original_dark', img_dark)
cv2.imshow('adjusted_dark', adjusted_dark)

cv2.imshow('original_light', img_light)
cv2.imshow('adjusted_light', adjusted_light)

cv2.waitKey(0)
#cv2.destroyAllWindows()

### 3. Aufnahme eines Weißbildes

### 4. Pixelfehler

In [ ]:
def get_hot_pixels():
    dark_img = get_image("./data/dark")
    return np.any(dark_img[:, :] > 0)


def get_stuck_pixels():
    dark_img = get_image("./data/dark")
    light_img = get_image("./data/light")
    return np.any(dark_img[:, :] == light_img[:, :])


def get_dead_pixels():
    light_img = get_image("./data/light")
    return np.any(light_img[:, :] == 0)


def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


print(get_hot_pixels())
print(get_stuck_pixels())
print(get_dead_pixels())